In [1]:
import pandas as pd
from sklearn import preprocessing
from neupy import environment
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from neupy import environment, algorithms, layers, estimators
from tqdm import tqdm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

%matplotlib inline

### Scale

In [2]:
training_df = pd.read_csv("train_70.0.csv").iloc[:,3:]
test_df = pd.read_csv("test_30.0.csv").iloc[:,3:]

from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = StandardScaler()
# scaler = MinMaxScaler()
training_df_scaled = pd.DataFrame(scaler.fit_transform(training_df), columns = training_df.columns)
testing_df_scaled = pd.DataFrame(scaler.transform(test_df), columns = training_df.columns)

In [3]:
training_X = training_df_scaled.iloc[:,:-1]
training_X.head(2)

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,...,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
0,-0.660716,0.411964,-0.013606,-0.001205,-0.001629,-0.165808,-0.858748,-0.546889,-0.061023,0.141094,...,0.679158,0.073029,0.172829,0.079331,-0.268003,0.252875,1.903209,2.732891,0.306378,2.830096
1,0.751215,0.154799,-0.012347,-0.001205,-0.005231,-0.522286,0.183354,0.534771,-0.300340,0.199287,...,0.063891,0.073029,0.978969,0.192025,-0.612844,0.080335,-0.562056,0.483741,0.306378,0.191135


In [4]:
training_Y = training_df_scaled.iloc[:,-1:]
training_Y.head(2)

,shares
0,0.039067
1,-0.171517


In [5]:
testing_X = testing_df_scaled.iloc[:,:-1]
testing_X.head(2)

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,...,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
0,0.751215,0.987929,-0.031659,-0.001205,-0.038704,-0.076688,0.704404,3.419196,-0.061023,0.128671,...,-0.190675,0.073029,0.172829,0.773383,0.076838,0.376118,-0.870215,-0.265976,0.837147,-0.688518
1,-0.660716,-0.100240,0.003443,-0.001205,0.010647,-0.165808,-0.858748,-0.426704,-0.061023,0.034548,...,0.633890,0.590033,0.978969,0.798475,0.938940,0.080335,-0.870215,-0.265976,0.837147,-0.688518


In [6]:
testing_Y = testing_df_scaled.iloc[:, -1:]
testing_Y.head(2)

,shares
0,-0.043336
1,-0.189829


### summary

In [7]:
training_X.iloc[:, :20].describe()

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min
count,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04
mean,1.034448e-16,6.606127e-17,6.145234e-18,6.145234e-18,1.126626e-17,-1.536309e-17,-1.536309e-17,-3.635930e-17,1.574716e-17,4.000804e-16,1.830768e-16,7.195045e-17,3.072617e-17,-1.216244e-17,2.534909e-17,-1.792360e-17,-2.304463e-17,-3.559115e-17,-1.382678e-17,-1.070295e-16
std,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00
min,-3.955223e+00,-1.169282e+00,-1.319704e-01,-1.611659e-01,-1.784229e-01,-9.678849e-01,-8.587478e-01,-5.468887e-01,-3.003397e-01,-5.375803e+00,-3.264444e+00,-2.359129e-01,-4.668046e-01,-4.326853e-01,-2.519992e-01,-4.774206e-01,-5.203083e-01,-3.868126e-01,-3.048287e-01,-4.969156e-01
25%,-6.607161e-01,-6.443250e-01,-2.018381e-02,-1.204752e-03,-1.801724e-02,-6.114061e-01,-5.982225e-01,-4.267044e-01,-3.003397e-01,-8.001540e-02,-6.460574e-01,-2.359129e-01,-4.668046e-01,-4.326853e-01,-2.519992e-01,-4.774206e-01,-5.203083e-01,-3.868126e-01,-1.873932e-01,-2.710931e-01
50%,-1.900723e-01,-2.893948e-01,-3.945246e-03,-1.204752e-03,-1.391169e-03,-2.549273e-01,-7.717183e-02,-4.267044e-01,-3.003397e-01,1.372914e-01,-1.223801e-01,-2.359129e-01,-4.668046e-01,-4.326853e-01,-2.519992e-01,-4.774206e-01,-5.203083e-01,-3.868126e-01,-1.312967e-01,-1.237429e-01
75%,7.512154e-01,3.630817e-01,1.262313e-02,-1.204752e-03,1.496480e-02,2.797909e-01,1.833535e-01,-6.615122e-02,-6.102349e-02,3.633936e-01,9.249746e-01,-2.359129e-01,-4.668046e-01,-4.326853e-01,-2.519992e-01,-4.774206e-01,-5.203083e-01,-3.146069e-01,-4.269579e-02,6.725603e-02
max,5.928298e+00,1.410122e+01,1.664915e+02,1.665183e+02,1.664498e+02,2.612450e+01,2.936219e+01,1.483671e+01,2.147744e+01,4.130836e+00,1.448652e+00,4.238853e+00,2.142224e+00,2.311149e+00,3.968266e+00,2.094589e+00,1.921937e+00,5.071943e+00,7.791563e+01,6.250750e+01


In [8]:
training_X.iloc[:, 21:40].describe()

,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02
count,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04
mean,4.455295e-17,-4.916188e-17,-2.355673e-17,7.476702e-17,-9.678744e-17,1.133028e-17,3.098222e-17,2.944592e-18,1.664334e-18,1.211123e-16,1.536309e-17,5.223449e-17,1.126626e-17,8.193646e-18,2.202042e-17,-2.355673e-17,-1.743710e-16,-5.377080e-17,-6.606127e-17
std,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00
min,-3.530775e+00,-1.927846e+00,-9.855136e-01,-9.154066e-01,-2.355835e+00,-2.114493e-01,-2.463818e-01,-2.715480e-01,-4.525985e-01,-4.783309e-01,-4.792975e-01,-4.706933e-01,-4.112301e-01,-2.577719e-01,-2.714934e-01,-3.881860e-01,-7.034875e-01,-6.451941e-01,-7.662924e-01
25%,4.211205e-01,-6.390002e-01,-9.846335e-01,-3.398842e-01,-5.664062e-01,-1.761761e-01,-2.202376e-01,-2.290611e-01,-4.525985e-01,-4.783309e-01,-4.792975e-01,-4.706933e-01,-4.112301e-01,-2.577719e-01,-2.714934e-01,-3.881860e-01,-6.079926e-01,-5.314007e-01,-6.650683e-01
50%,4.211205e-01,-1.105219e-01,-7.562646e-02,-2.123717e-01,-2.008446e-01,-1.453120e-01,-1.798329e-01,-1.766534e-01,-4.525985e-01,-4.783309e-01,-4.792975e-01,-4.706933e-01,-4.112301e-01,-2.577719e-01,-2.714934e-01,-3.881860e-01,-5.761795e-01,-4.934824e-01,-6.245663e-01
75%,4.211205e-01,5.312089e-01,8.223448e-01,5.576551e-02,3.448509e-01,-6.815178e-02,-5.861879e-02,-4.940829e-02,-4.525985e-01,-4.783309e-01,-4.792975e-01,-4.706933e-01,-4.112301e-01,-2.577719e-01,-2.714934e-01,-3.881860e-01,2.199732e-01,4.468763e-02,4.147575e-01
max,4.211205e-01,4.351808e+00,2.195068e+00,4.726025e+01,2.587878e+01,3.783955e+01,1.979673e+01,2.950811e+01,2.209464e+00,2.090603e+00,2.086387e+00,2.124526e+00,2.431728e+00,3.879399e+00,3.683331e+00,2.576085e+00,2.830838e+00,3.567643e+00,2.493114e+00


In [9]:
training_X.iloc[:, 41:].describe()

,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
count,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04,2.775000e+04
mean,8.040015e-17,2.984279e-16,1.446691e-17,1.817325e-16,-2.122666e-16,3.447733e-16,1.173996e-16,1.597761e-16,-3.231369e-16,3.592082e-16,-8.116831e-17,1.958793e-17,7.246256e-17,-8.961800e-18,8.193646e-18,-9.217852e-17,7.393485e-17
std,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00
min,-8.097722e-01,-3.802284e+00,-5.300707e+00,-2.280890e+00,-1.543111e+00,-3.585024e+00,-1.844727e+00,-3.379258e+00,-1.348732e+00,-3.051733e+00,-5.778505e+00,-1.647367e+00,-9.236829e+00,-8.702146e-01,-4.014559e+00,-1.816698e+00,-6.885182e-01
25%,-7.111149e-01,-4.021992e-01,-6.330887e-01,-6.463869e-01,-6.481381e-01,-4.316234e-01,-6.576720e-01,-4.517814e-01,-6.378517e-01,-6.333119e-01,-5.323186e-01,-6.128440e-01,-1.784751e-01,-8.702146e-01,-2.659758e-01,-9.320832e-01,-6.885182e-01
50%,-6.688256e-01,8.817498e-02,-1.827611e-03,-3.035204e-02,-1.184606e-01,1.492662e-01,-5.367061e-02,4.754101e-02,7.302890e-02,1.728285e-01,4.851900e-02,7.683772e-02,8.033497e-02,-4.336572e-01,-2.659758e-01,8.371472e-01,-6.885182e-01
75%,5.711956e-01,5.530251e-01,6.026144e-01,6.128189e-01,4.803053e-01,6.195101e-01,6.124765e-01,5.471467e-01,7.302890e-02,9.789690e-01,5.642801e-01,7.665195e-01,5.979552e-01,6.705763e-01,2.803376e-01,8.371472e-01,4.110489e-01
max,2.391560e+00,4.772566e+00,5.536088e+00,6.672846e+00,1.566978e+01,1.670644e+00,4.565369e+00,6.180725e+00,1.286888e+01,9.789690e-01,2.028000e+00,1.801042e+00,1.115575e+00,2.211367e+00,3.482608e+00,8.371472e-01,3.709750e+00


### MLP

In [10]:
a = np.random.randn(5)
b = np.random.randn(5)
estimators.rmse(a, b) == math.sqrt(mean_squared_error(a, b))

True

In [11]:
from sklearn.neural_network import MLPRegressor
import math 

result = []
for thisactivation in tqdm(['tanh', 'relu', 'logistic']):
    for thisalpha in [0.0001, 1.0, 100]:
        for sz in [10, 50, 100]:
            mlpreg = MLPRegressor(hidden_layer_sizes= [sz,sz,sz],
                                 activation=thisactivation,
                                 alpha=thisalpha, solver='lbfgs').fit(training_X, training_Y)
            
            y_pred = mlpreg.predict(testing_X)
            result.append([sz, thisalpha, thisactivation, 
                           math.sqrt(mean_squared_error(testing_Y, y_pred)),
                          mean_absolute_error(testing_Y, y_pred)])

  0%|          | 0/3 [00:00<?, ?it/s]/home/lam/anaconda2/envs/neupy/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
100%|██████████| 3/3 [08:58<00:00, 179.35s/it]


In [12]:
results_mlp = pd.DataFrame(result, columns=['size x size', 'alpha', 'activation', 'RMSE', 'MAE'])

In [13]:
results_mlp.sort_values(by=['RMSE'], ascending=[1]).head(5)

,size x size,alpha,activation,RMSE,MAE
7,50,100.0000,tanh,1.188981,0.283053
8,100,100.0000,tanh,1.189094,0.283723
23,100,1.0000,logistic,1.189412,0.282226
20,100,0.0001,logistic,1.190480,0.284598
15,10,100.0000,relu,1.191322,0.287934


In [14]:
results_mlp.sort_values(by=['MAE'], ascending=[1]).head(5)

,size x size,alpha,activation,RMSE,MAE
23,100,1.0000,logistic,1.189412,0.282226
7,50,100.0000,tanh,1.188981,0.283053
8,100,100.0000,tanh,1.189094,0.283723
20,100,0.0001,logistic,1.190480,0.284598
13,50,1.0000,relu,1.220039,0.285595


### GRNN

In [15]:
environment.reproducible()

In [19]:
nw = algorithms.GRNN(std=1, verbose=False)
nw.train(training_X, training_Y)
y_predicted = nw.predict(testing_X)

In [20]:
print('RMSE', math.sqrt(mean_squared_error(testing_Y, y_predicted)))
print('MAE', mean_absolute_error(testing_Y, y_predicted))

RMSE 1.2767123860148182
MAE 0.313673651996
